## Description:
Extracts an HDF5 file of the entire field from the imported nek output file.

In [1]:
from __future__ import annotations
from pathlib import Path
import os
os.chdir("../../..")
import yaml
from flamekit.datasets import extract_full_field_hdf5
from flamekit.io_fronts import Case
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank


### Read config file and define helper functions

In [2]:
PROJECT_ROOT = Path.cwd().resolve()
CONFIG_PATH = PROJECT_ROOT / "notebooks" / "preprocessing/extract_fields" / "extract_fields.yaml"
CFG = yaml.safe_load(CONFIG_PATH.read_text(encoding="utf-8"))

DATA_BASE_DIR = PROJECT_ROOT / Path(CFG["DATA_BASE_DIR"])
OUTPUT_BASE_DIR = PROJECT_ROOT / Path(CFG["OUTPUT_BASE_DIR"])  
POST = bool(CFG["POST"])
FILE_NAME = "po_postPremix" if POST else "premix"

TIME_STEP = int(CFG["TIME_STEP"])
MULTI_TIME_STEP = bool(CFG["MULTI_TIME_STEP"])
TIME_STEP_START = int(CFG["TIME_STEP_START"])
TIME_STEP_END = int(CFG["TIME_STEP_END"])

PHI = float(CFG["PHI"])
LAT_SIZE = str(CFG["LAT_SIZE"])
T_REF = float(CFG["T_REF"])
P_REF = float(CFG["P_REF"])

SCALARS = list(CFG["SCALARS"])

COMP_VEL_JACOBIAN = bool(CFG["COMP_VEL_JACOBIAN"])
COMP_VEL_HESSIAN = bool(CFG["COMP_VEL_HESSIAN"])
COMP_REACTION_RATES = bool(CFG["COMP_REACTION_RATES"])
COMP_T_GRAD = bool(CFG["COMP_T_GRAD"])
COMP_CURV_GRAD = bool(CFG["COMP_CURV_GRAD"])
COMP_LOCAL_VEL_JACOBIAN = bool(CFG["COMP_LOCAL_VEL_JACOBIAN"])

CANTERA_YAML = PROJECT_ROOT / Path(CFG["CANTERA_YAML"])
CANTERA_INPUTS = [str(CANTERA_YAML), None, T_REF, P_REF]

def _process_one_timestep(ts: int) -> None:
    case = Case(
        base_dir=OUTPUT_BASE_DIR,
        phi=PHI,
        lat_size=LAT_SIZE,
        time_step=ts,
        post=POST,
    )

    out_path = extract_full_field_hdf5(
        case,
        data_base_dir=DATA_BASE_DIR,
        output_base_dir=OUTPUT_BASE_DIR,
        file_name=FILE_NAME,
        scalar_names=SCALARS,
        comm=comm,
        compute_vel_jacobian=COMP_VEL_JACOBIAN,
        compute_vel_hessian=COMP_VEL_HESSIAN,
        compute_reaction_rates=COMP_REACTION_RATES,
        compute_T_grad=COMP_T_GRAD,
        compute_curv_grad=COMP_CURV_GRAD,
        compute_local_vel_jacobian=COMP_LOCAL_VEL_JACOBIAN,
        compute_progress_var=True,
        cantera_inputs=CANTERA_INPUTS,
    )

    if rank == 0:
        print(f"[rank0] Saved full field: {out_path}")


### Run

In [3]:
if MULTI_TIME_STEP:
    for ts in range(TIME_STEP_START, TIME_STEP_END + 1):
        if rank == 0:
            print(f"=== Processing TIME_STEP={ts} ===")
        _process_one_timestep(ts)
else:
    if rank == 0:
        print(f"=== Processing TIME_STEP={TIME_STEP} ===")
    _process_one_timestep(TIME_STEP)  

=== Processing TIME_STEP=249 ===
2026-01-17 21:53:07,622 - Mesh - INFO - Initializing empty Mesh object.
2026-01-17 21:53:07,622 - Field - INFO - Initializing empty Field object
2026-01-17 21:53:07,623 - pynekread - INFO - Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/nek/phi0.40/h400x025_ref/premix0.f00001
2026-01-17 21:53:07,636 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2026-01-17 21:53:07,636 - Mesh - INFO - Initializing common attributes.
2026-01-17 21:53:07,637 - Mesh - INFO - Getting vertices
2026-01-17 21:53:07,638 - Mesh - INFO - Getting edge centers
2026-01-17 21:53:07,666 - Mesh - INFO - Facet centers not available for 2D
2026-01-17 21:53:07,667 - Mesh - INFO - Mesh object initialized.
2026-01-17 21:53:07,667 - Mesh - INFO - Mesh data is of type: float64
2026-01-17 21:53:07,668 - Mesh - INFO - Elapsed time: 0.031983445s
2026-01-17 21:53:07,668 - pynekread - INFO - Reading field data
2026-01-17 21:53:07,668 - pynekread -

In [4]:
# Plot a variable over the full field
import matplotlib.pyplot as plt
from flamekit.io_fields import load_field_hdf5

PLOT_VAR = "HRR"
MAX_FIELD_POINTS = 200000

plot_ts = TIME_STEP_END if MULTI_TIME_STEP else TIME_STEP
plot_case = Case(
    base_dir=OUTPUT_BASE_DIR,
    phi=PHI,
    lat_size=LAT_SIZE,
    time_step=plot_ts,
    post=POST,
)

if rank == 0:
    field_df = load_field_hdf5(plot_case, required_cols=["x", "y", PLOT_VAR])
    if len(field_df) > MAX_FIELD_POINTS:
        field_df = field_df.sample(MAX_FIELD_POINTS, random_state=42)

    fig, ax = plt.subplots(figsize=(7.2, 6.2), dpi=200, constrained_layout=True)
    sc = ax.scatter(
        field_df["x"],
        field_df["y"],
        c=field_df[PLOT_VAR],
        s=2,
        cmap="viridis",
    )
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(f"{PLOT_VAR} at t={plot_ts}")
    plt.colorbar(sc, ax=ax, label=PLOT_VAR)
    plt.show()
    plt.close(fig)


ValueError: Missing columns ['HRR'] in extracted_field_300.hdf5